# Individual Project
By Jeff Akins

#### ![EV Charging Station](https://i.insider.com/6042a8ac44d8e300117b9655?width=1200&format=jpeg)

## Exploring the State of the Electric Vehicle Charging Network within the U.S.
The world's methods of transportation are evolving, not only here in the United States, but globally. The EU recently proposed a mandate that all new vehicles be emissions free after 2035. In the U.S., the Biden administrations wants half of all new vehicles to be purely electric by 2030. Regardless of the mandates, buyers may be choosing to purchase EVs over their traditional internal combustion engine (ICE) counterparts in the upcoming years. No longer do you have to choose between the environment or the car that you want to drive. Many new EVs are surpassing comparable ICE vehicles in both speed, handling, and overall capability and new models of EVs are being introduced on a regular basis.

So, with all these new EV coming online, is the charging infrastructure keeping up? Buyers may be asking themselves, how many charging stations will be nearby if I need one? If I decide to go on a road trip, how difficult will it be to find an open charging station? I will try and answer these questions using a bit of machine learning to show the state of the charging networks now and the predicted growth over the next several years.

So, let's begin...

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

import wrangle as w
import warnings
warnings.filterwarnings('ignore')

##### You can find the original dataset file [here](https://catalog.data.gov/dataset/alternative-fueling-station-locations-422f2/resource/341957d8-daf6-4a38-ab1d-8ec1bc21cfb9) on data.gov under the name 'alternative-fuel-stations-2021-csv-3.csv'

In [2]:
csv_file_name = 'alternative-fuel-stations-2021-csv-3.csv'
ev = w.clean_ev(csv_file_name)

In [3]:
ev.head()

,fuel_code,station_name,address,city,state,zip,group_access_code,level1_evse_num,level2_evse_num,ev_dc_fast_count,ev_network,lat,long,id,update_date,open_date,connector_type,country,pricing,year_opened
159,ELEC,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,Private,0,39,3,Non-Networked,34.248319,-118.387971,1517,2021-07-14,1999-10-15,CHADEMO J1772 J1772COMBO,US,Cost,1999
160,ELEC,LADWP - West LA District Office,1394 S Sepulveda Blvd,Los Angeles,CA,90024,Private,0,4,0,Non-Networked,34.052542,-118.448504,1519,2021-07-14,2020-02-28,J1772,US,Free,2020
161,ELEC,Southern California Edison - Rosemead Office B...,2244 Walnut Grove Ave,Rosemead,CA,91770,TEMPORARILY UNAVAILABLE (Private),22,22,0,Greenlots,34.050745,-118.081014,1520,2021-03-11,2011-08-01,J1772 NEMA520,US,Cost,2011
162,ELEC,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,Public,0,12,0,Non-Networked,34.040539,-118.271387,1523,2021-03-11,1995-08-30,J1772,US,Cost,1995
163,ELEC,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,Private,0,311,2,Non-Networked,34.059133,-118.248589,1525,2021-07-14,1999-10-15,CHADEMO J1772 J1772COMBO,US,Cost,1999


In [4]:
ev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47086 entries, 159 to 56793
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fuel_code          47086 non-null  object        
 1   station_name       47086 non-null  object        
 2   address            47086 non-null  object        
 3   city               47086 non-null  object        
 4   state              47086 non-null  object        
 5   zip                47086 non-null  object        
 6   group_access_code  47086 non-null  object        
 7   level1_evse_num    47086 non-null  int64         
 8   level2_evse_num    47086 non-null  int64         
 9   ev_dc_fast_count   47086 non-null  int64         
 10  ev_network         47086 non-null  object        
 11  lat                47086 non-null  float64       
 12  long               47086 non-null  float64       
 13  id                 47086 non-null  int64         
 14  upda